Let's load the data from the csv just as in `dataset.ipynb`.

In [ ]:
import pandas as pd
import numpy as np

raw_data_file_name = "../dataset/fer2013.csv"
raw_data = pd.read_csv(raw_data_file_name)

In [ ]:
import keras

train_data = raw_data[raw_data["Usage"] == "Training"]
train_expected = keras.utils.to_categorical(train_data["emotion"], num_classes=7, dtype='int32')
first_test_data = raw_data[raw_data["Usage"] == "PrivateTest"]
first_test_expected = keras.utils.to_categorical(first_test_data["emotion"], num_classes=7, dtype='int32')

In [ ]:
def process_pixel_row(row):
    split = row.split(" ")
    pixels = np.array(split, 'float32')
    pixels /= 255
    return pixels.reshape(48, 48)

In [ ]:
train_pixels = train_data["pixels"].apply(process_pixel_row)

In [ ]:
first_test_pixels = first_test_data["pixels"].apply(process_pixel_row)

In [ ]:
def duplicate_input_layer(array_input):
    vg_input = np.empty([int(len(array_input)), 48, 48, 3])
    for index, item in enumerate(vg_input):
        item[:, :, 0] = array_input[index]
        item[:, :, 1] = array_input[index]
        item[:, :, 2] = array_input[index]
    return vg_input

train_pixels_duplicated = duplicate_input_layer(train_pixels)

Now, let's load up a pre-trained neural network from `keras`, a high-level API on TensorFlow.

In [ ]:
vgg16 = keras.applications.vgg16.VGG16(include_top=False, input_shape=(48, 48, 3), weights='imagenet')

In [ ]:
train_output_features = vgg16.predict(train_pixels_duplicated)

In [ ]:
train_feature_map = np.empty([len(train_output_features), 512])
for idx_pic, picture in enumerate(train_output_features):
    train_feature_map[idx_pic] = picture

Here, we create our own top-level network to load on top of VGG16.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

topLayerModel = Sequential()
topLayerModel.add(Dense(256, input_shape=(512,), activation='relu'))
topLayerModel.add(Dense(256, input_shape=(256,), activation='relu'))
topLayerModel.add(Dropout(0.5))
topLayerModel.add(Dense(128, input_shape=(256,), activation='relu'))
topLayerModel.add(Dense(7, activation='softmax'))

In [ ]:
from keras import optimizers

adamax = optimizers.Adamax()

topLayerModel.compile(loss='categorical_crossentropy',
              optimizer=adamax, metrics=['accuracy'])

topLayerModel.fit(train_feature_map, train_expected,
          validation_data=(train_feature_map, train_expected),
          epoch=1, batch_size=32)